# Welcome back

Welcome back! In our third session, we will do the following:

| What | How long |
|:------------------------------------------------------------------------|--------------:|
| Data frames refresh and fetch triangle data | 10 min |
| Indexing | 10 min.
| Querying a data frame | 15 min |
| Packing/unpacking | 15 min |
| Grouped operations | 20 min | 
| I/O | 5 min |
| Recap on iterators | 15 min |

# Data frames refresh

In [27]:
import pandas as pd
import  numpy as np

In [2]:
url = 'https://www.casact.org/research/reserve_data/wkcomp_pos.csv'
df_triangle = pd.read_csv(url)

df_triangle.head()

,GRCODE,GRNAME,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_D,CumPaidLoss_D,BulkLoss_D,EarnedPremDIR_D,EarnedPremCeded_D,EarnedPremNet_D,Single,PostedReserve97_D
0,86,Allstate Ins Co Grp,1988,1988,1,367404,70571,127737,400699,5957,394742,0,281872
1,86,Allstate Ins Co Grp,1988,1989,2,362988,155905,60173,400699,5957,394742,0,281872
2,86,Allstate Ins Co Grp,1988,1990,3,347288,220744,27763,400699,5957,394742,0,281872
3,86,Allstate Ins Co Grp,1988,1991,4,330648,251595,15280,400699,5957,394742,0,281872
4,86,Allstate Ins Co Grp,1988,1992,5,354690,274156,27689,400699,5957,394742,0,281872


In [3]:
df_triangle['GRNAME'].head(), df_triangle['GRNAME'].tail()

(0    Allstate Ins Co Grp
 1    Allstate Ins Co Grp
 2    Allstate Ins Co Grp
 3    Allstate Ins Co Grp
 4    Allstate Ins Co Grp
 Name: GRNAME, dtype: object,
 13195    Tower Ins Co Of NY
 13196    Tower Ins Co Of NY
 13197    Tower Ins Co Of NY
 13198    Tower Ins Co Of NY
 13199    Tower Ins Co Of NY
 Name: GRNAME, dtype: object)

In [4]:
type(df_triangle['GRNAME'])

pandas.core.series.Series

In [5]:
df_triangle.columns

Index(['GRCODE', 'GRNAME', 'AccidentYear', 'DevelopmentYear', 'DevelopmentLag',
       'IncurLoss_D', 'CumPaidLoss_D', 'BulkLoss_D', 'EarnedPremDIR_D',
       'EarnedPremCeded_D', 'EarnedPremNet_D', 'Single', 'PostedReserve97_D'],
      dtype='object')

In [6]:
df_triangle.shape

(13200, 13)

By the by, we get back a tuple from `shape`.

In [7]:
df_triangle.dtypes

GRCODE                int64
GRNAME               object
AccidentYear          int64
DevelopmentYear       int64
DevelopmentLag        int64
IncurLoss_D           int64
CumPaidLoss_D         int64
BulkLoss_D            int64
EarnedPremDIR_D       int64
EarnedPremCeded_D     int64
EarnedPremNet_D       int64
Single                int64
PostedReserve97_D     int64
dtype: object

Note that the financial amounts are integers. This is great if we're using a Poisson GLM> 

In [8]:
df_triangle.describe().round(2)

,GRCODE,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_D,CumPaidLoss_D,BulkLoss_D,EarnedPremDIR_D,EarnedPremCeded_D,EarnedPremNet_D,Single,PostedReserve97_D
count,13200.00,13200.00,13200.00,13200.00,13200.00,13200.00,13200.00,13200.00,13200.00,13200.00,13200.00,13200.00
mean,17153.05,1992.50,1997.00,5.50,11532.05,8215.74,1570.13,18438.47,1812.34,16626.13,0.73,39714.40
std,12512.21,2.87,4.06,2.87,35595.56,25714.08,7259.02,51830.70,6666.66,48941.72,0.45,130130.68
min,86.00,1988.00,1988.00,1.00,-59.00,-338.00,-4621.00,-6518.00,-3522.00,-9731.00,0.00,0.00
25%,8526.00,1990.00,1994.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,411.00
50%,14110.00,1992.50,1997.00,5.50,544.00,351.50,5.00,1419.00,144.50,827.00,1.00,2732.00
75%,26983.25,1995.00,2000.00,8.00,6526.50,4565.00,259.25,11354.25,1141.00,9180.50,1.00,19265.75
max,44300.00,1997.00,2006.00,10.00,367404.00,325322.00,145296.00,421223.00,78730.00,418755.00,1.00,1090093.00


## Renaming columns

In [9]:
new_names = {
  'CumPaidLoss_D': 'cumulative_paid',
  'IncurLoss_D': 'cumulative_incurred',
}
df_triangle = df_triangle.rename(columns = new_names)
df_triangle.columns

Index(['GRCODE', 'GRNAME', 'AccidentYear', 'DevelopmentYear', 'DevelopmentLag',
       'cumulative_incurred', 'cumulative_paid', 'BulkLoss_D',
       'EarnedPremDIR_D', 'EarnedPremCeded_D', 'EarnedPremNet_D', 'Single',
       'PostedReserve97_D'],
      dtype='object')

## Synthesizing new data

In [10]:
df_triangle['paid_to_incurred'] = df_triangle['cumulative_paid'] / df_triangle['cumulative_incurred']
df_triangle[['cumulative_paid', 'cumulative_incurred', 'paid_to_incurred']].head()

,cumulative_paid,cumulative_incurred,paid_to_incurred
0,70571,367404,0.192080
1,155905,362988,0.429505
2,220744,347288,0.635622
3,251595,330648,0.760915
4,274156,354690,0.772945


In [63]:
import numpy as np

def div0( a, b ):
    """ ignore / 0, div0( [-1, 0, 1], 0 ) -> [0, 0, 0] """
    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.true_divide( a, b )
        c[ ~ np.isfinite( c )] = 0  # -inf inf NaN
    return c

df_triangle['paid_to_incurred'] = div0( df_triangle['cumulative_paid'], df_triangle['cumulative_incurred'] )

In [67]:
import division_by_zero as dv
df_triangle['paid_to_incurred'] = dv.div0( df_triangle['cumulative_paid'], df_triangle['cumulative_incurred'] )

ModuleNotFoundError: No module named 'division_by_zero'

In [71]:
# Remove dividing by zero
df_triangle['paid_to_incurred'] =  df_triangle['cumulative_paid'] / df_triangle['cumulative_incurred']
df_triangle['paid_to_incurred'] = df_triangle['paid_to_incurred'] .apply(lambda x: 0 if np.isinf(x) else x)

In [66]:
import os 
os.getcwd()

'C:\\Users\\e015614\\Documents\\GitHub\\CAS-Python-Intro\\CAS Python Course'

In [42]:
df_triangle['paid_to_incurred'].describe()

count    13200.000000
mean         0.515107
std          0.427412
min         -3.333333
25%          0.000000
50%          0.638353
75%          0.931815
max          3.000000
Name: paid_to_incurred, dtype: float64

In [72]:
df_triangle.head()

,GRCODE,AccidentYear,DevelopmentYear,DevelopmentLag,cumulative_incurred,cumulative_paid,BulkLoss_D,EarnedPremDIR_D,EarnedPremCeded_D,EarnedPremNet_D,Single,PostedReserve97_D,paid_to_incurred
GRNAME,,,,,,,,,,,,,
Allstate Ins Co Grp,86,1988,1988,1,367404,70571,127737,400699,5957,394742,0,281872,0.192080
Allstate Ins Co Grp,86,1988,1989,2,362988,155905,60173,400699,5957,394742,0,281872,0.429505
Allstate Ins Co Grp,86,1988,1990,3,347288,220744,27763,400699,5957,394742,0,281872,0.635622
Allstate Ins Co Grp,86,1988,1991,4,330648,251595,15280,400699,5957,394742,0,281872,0.760915
Allstate Ins Co Grp,86,1988,1992,5,354690,274156,27689,400699,5957,394742,0,281872,0.772945


In [13]:
df_triangle['paid_to_incurred'] = df_triangle.cumulative_paid / df_triangle.cumulative_incurred
df_triangle['paid_to_incurred'].describe()

count    9251.000000
mean            -inf
std              NaN
min             -inf
25%         0.575371
50%         0.868966
75%         0.965742
max         3.000000
Name: paid_to_incurred, dtype: float64

# Indexing

Pandas pays a lot more attention to indices than data frames in R. 

In [73]:
df_triangle.index

Index(['Allstate Ins Co Grp', 'Allstate Ins Co Grp', 'Allstate Ins Co Grp',
       'Allstate Ins Co Grp', 'Allstate Ins Co Grp', 'Allstate Ins Co Grp',
       'Allstate Ins Co Grp', 'Allstate Ins Co Grp', 'Allstate Ins Co Grp',
       'Allstate Ins Co Grp',
       ...
       'Tower Ins Co Of NY', 'Tower Ins Co Of NY', 'Tower Ins Co Of NY',
       'Tower Ins Co Of NY', 'Tower Ins Co Of NY', 'Tower Ins Co Of NY',
       'Tower Ins Co Of NY', 'Tower Ins Co Of NY', 'Tower Ins Co Of NY',
       'Tower Ins Co Of NY'],
      dtype='object', name='GRNAME', length=13200)

In [15]:
df_triangle = df_triangle.set_index('GRNAME')
df_triangle.index

Index(['Allstate Ins Co Grp', 'Allstate Ins Co Grp', 'Allstate Ins Co Grp',
       'Allstate Ins Co Grp', 'Allstate Ins Co Grp', 'Allstate Ins Co Grp',
       'Allstate Ins Co Grp', 'Allstate Ins Co Grp', 'Allstate Ins Co Grp',
       'Allstate Ins Co Grp',
       ...
       'Tower Ins Co Of NY', 'Tower Ins Co Of NY', 'Tower Ins Co Of NY',
       'Tower Ins Co Of NY', 'Tower Ins Co Of NY', 'Tower Ins Co Of NY',
       'Tower Ins Co Of NY', 'Tower Ins Co Of NY', 'Tower Ins Co Of NY',
       'Tower Ins Co Of NY'],
      dtype='object', name='GRNAME', length=13200)

An index may contain multiple values

In [16]:
# df_triangle.reset_index(inplace=True)
# df_triangle.index

In [74]:
df_triangle = df_triangle.set_index(['GRNAME', 'AccidentYear', 'DevelopmentYear'])

KeyError: "None of ['GRNAME'] are in the columns"

In [75]:
df_triangle.columns

Index(['GRCODE', 'AccidentYear', 'DevelopmentYear', 'DevelopmentLag',
       'cumulative_incurred', 'cumulative_paid', 'BulkLoss_D',
       'EarnedPremDIR_D', 'EarnedPremCeded_D', 'EarnedPremNet_D', 'Single',
       'PostedReserve97_D', 'paid_to_incurred'],
      dtype='object')

In [76]:
df_triangle.head()

,GRCODE,AccidentYear,DevelopmentYear,DevelopmentLag,cumulative_incurred,cumulative_paid,BulkLoss_D,EarnedPremDIR_D,EarnedPremCeded_D,EarnedPremNet_D,Single,PostedReserve97_D,paid_to_incurred
GRNAME,,,,,,,,,,,,,
Allstate Ins Co Grp,86,1988,1988,1,367404,70571,127737,400699,5957,394742,0,281872,0.192080
Allstate Ins Co Grp,86,1988,1989,2,362988,155905,60173,400699,5957,394742,0,281872,0.429505
Allstate Ins Co Grp,86,1988,1990,3,347288,220744,27763,400699,5957,394742,0,281872,0.635622
Allstate Ins Co Grp,86,1988,1991,4,330648,251595,15280,400699,5957,394742,0,281872,0.760915
Allstate Ins Co Grp,86,1988,1992,5,354690,274156,27689,400699,5957,394742,0,281872,0.772945


In [77]:
df_triangle = df_triangle.set_index(['AccidentYear', 'DevelopmentYear'], append = True)
df_triangle.index

MultiIndex([('Allstate Ins Co Grp', 1988, 1988),
            ('Allstate Ins Co Grp', 1988, 1989),
            ('Allstate Ins Co Grp', 1988, 1990),
            ('Allstate Ins Co Grp', 1988, 1991),
            ('Allstate Ins Co Grp', 1988, 1992),
            ('Allstate Ins Co Grp', 1988, 1993),
            ('Allstate Ins Co Grp', 1988, 1994),
            ('Allstate Ins Co Grp', 1988, 1995),
            ('Allstate Ins Co Grp', 1988, 1996),
            ('Allstate Ins Co Grp', 1988, 1997),
            ...
            ( 'Tower Ins Co Of NY', 1997, 1997),
            ( 'Tower Ins Co Of NY', 1997, 1998),
            ( 'Tower Ins Co Of NY', 1997, 1999),
            ( 'Tower Ins Co Of NY', 1997, 2000),
            ( 'Tower Ins Co Of NY', 1997, 2001),
            ( 'Tower Ins Co Of NY', 1997, 2002),
            ( 'Tower Ins Co Of NY', 1997, 2003),
            ( 'Tower Ins Co Of NY', 1997, 2004),
            ( 'Tower Ins Co Of NY', 1997, 2005),
            ( 'Tower Ins Co Of NY', 1997, 2006)],
   

Index columns disappear from the data frame unless you explicitly tell pandas not to.

In [78]:
df_triangle.head()

GRCODE  DevelopmentLag  \
GRNAME              AccidentYear DevelopmentYear                           
Allstate Ins Co Grp 1988         1988                 86               1   
                                 1989                 86               2   
                                 1990                 86               3   
                                 1991                 86               4   
                                 1992                 86               5   

                                                  cumulative_incurred  \
GRNAME              AccidentYear DevelopmentYear                        
Allstate Ins Co Grp 1988         1988                          367404   
                                 1989                          362988   
                                 1990                          347288   
                                 1991                          330648   
                                 1992                          354690   

                                                  cumulative_paid  BulkLoss_D  \
GRNAME              AccidentYear DevelopmentYear                                
Allstate Ins Co Grp 1988         1988                       70571      127737   
                                 1989                      155905       60173   
                                 1990                      220744       27763   
                                 1991                      251595       15280   
                                 1992                      274156       27689   

                                                  EarnedPremDIR_D  \
GRNAME              AccidentYear DevelopmentYear                    
Allstate Ins Co Grp 1988         1988                      400699   
                                 1989                      400699   
                                 1990                      400699   
                                 1991                      400699   
                                 1992                      400699   

                                                  EarnedPremCeded_D  \
GRNAME              AccidentYear DevelopmentYear                      
Allstate Ins Co Grp 1988         1988                          5957   
                                 1989                          5957   
                                 1990                          5957   
                                 1991                          5957   
                                 1992                          5957   

                                                  EarnedPremNet_D  Single  \
GRNAME              AccidentYear DevelopmentYear                            
Allstate Ins Co Grp 1988         1988                      394742       0   
                                 1989                      394742       0   
                                 1990                      394742       0   
                                 1991                      394742       0   
                                 1992                      394742       0   

                                                  PostedReserve97_D  \
GRNAME              AccidentYear DevelopmentYear                      
Allstate Ins Co Grp 1988         1988                        281872   
                                 1989                        281872   
                                 1990                        281872   
                                 1991                        281872   
                                 1992                        281872   

                                                  paid_to_incurred  
GRNAME              AccidentYear DevelopmentYear                    
Allstate Ins Co Grp 1988         1988                     0.192080  
                                 1989                     0.429505  
                                 1990                     0.635622  
                                 1991                     0.760915  
   

In [79]:
df_triangle['lag'] = df_triangle['DevelopmentYear'] - df_allstate['AccidentYear'] + 1
#Development is an index not a column.


KeyError: 'DevelopmentYear'

A useful strategy is to carry out all of the non-indexed operations before creating the index.

In [81]:
df_triangle = df_triangle.reset_index()
df_triangle.index

RangeIndex(start=0, stop=13200, step=1)

In [83]:
df_triangle['lag'] = df_triangle['DevelopmentYear'] - df_triangle['AccidentYear'] + 1
df_triangle.lag
df_triangle['lagmo'] = (df_triangle['DevelopmentYear'] - df_triangle['AccidentYear'] + 1)*12

That's the only one we need for now, so we can go ahead and set an index.

In [85]:
df_triangle = df_triangle.set_index(['GRNAME', 'AccidentYear', 'lag'])

In [86]:
df_triangle.head()

DevelopmentYear  GRCODE  DevelopmentLag  \
GRNAME              AccidentYear lag                                            
Allstate Ins Co Grp 1988         1               1988      86               1   
                                 2               1989      86               2   
                                 3               1990      86               3   
                                 4               1991      86               4   
                                 5               1992      86               5   

                                      cumulative_incurred  cumulative_paid  \
GRNAME              AccidentYear lag                                         
Allstate Ins Co Grp 1988         1                 367404            70571   
                                 2                 362988           155905   
                                 3                 347288           220744   
                                 4                 330648           251595   
                                 5                 354690           274156   

                                      BulkLoss_D  EarnedPremDIR_D  \
GRNAME              AccidentYear lag                                
Allstate Ins Co Grp 1988         1        127737           400699   
                                 2         60173           400699   
                                 3         27763           400699   
                                 4         15280           400699   
                                 5         27689           400699   

                                      EarnedPremCeded_D  EarnedPremNet_D  \
GRNAME              AccidentYear lag                                       
Allstate Ins Co Grp 1988         1                 5957           394742   
                                 2                 5957           394742   
                                 3                 5957           394742   
                                 4                 5957           394742   
                                 5                 5957           394742   

                                      Single  PostedReserve97_D  \
GRNAME              AccidentYear lag                              
Allstate Ins Co Grp 1988         1         0             281872   
                                 2         0             281872   
                                 3         0             281872   
                                 4         0             281872   
                                 5         0             281872   

                                      paid_to_incurred  lagmo  
GRNAME              AccidentYear lag                           
Allstate Ins Co Grp 1988         1            0.192080     12  
                                 2            0.429505     24  
                                 3            0.635622     36  
                                 4            0.760915     48  
                                 5            0.772945     60

# Querying a data frame

## Columnar subsets

In [87]:
df_triangle[['cumulative_paid', 'cumulative_incurred', 'paid_to_incurred']]

cumulative_paid  cumulative_incurred  \
GRNAME              AccidentYear lag                                         
Allstate Ins Co Grp 1988         1              70571               367404   
                                 2             155905               362988   
                                 3             220744               347288   
                                 4             251595               330648   
                                 5             274156               354690   
...                                               ...                  ...   
Tower Ins Co Of NY  1997         6                287                  334   
                                 7                293                  318   
                                 8                300                  323   
                                 9                297                  310   
                                 10               297                  312   

                                      paid_to_incurred  
GRNAME              AccidentYear lag                    
Allstate Ins Co Grp 1988         1            0.192080  
                                 2            0.429505  
                                 3            0.635622  
                                 4            0.760915  
                                 5            0.772945  
...                                                ...  
Tower Ins Co Of NY  1997         6            0.859281  
                                 7            0.921384  
                                 8            0.928793  
                                 9            0.958065  
                                 10           0.951923  

[13200 rows x 3 columns]

Pass in a list with the names of columns to return.

In [89]:
my_cols = ['cumulative_paid', 'cumulative_incurred']
df_triangle[my_cols]

cumulative_paid  cumulative_incurred
GRNAME              AccidentYear lag                                      
Allstate Ins Co Grp 1988         1              70571               367404
                                 2             155905               362988
                                 3             220744               347288
                                 4             251595               330648
                                 5             274156               354690
...                                               ...                  ...
Tower Ins Co Of NY  1997         6                287                  334
                                 7                293                  318
                                 8                300                  323
                                 9                297                  310
                                 10               297                  312

[13200 rows x 2 columns]

Create a list using list comprehension

In [90]:
#List object
cumul_cols = [col for col in df_triangle.columns if 'cumul' in col]
cumul_cols

['cumulative_incurred', 'cumulative_paid']

In [91]:
df_triangle[cumul_cols]

cumulative_incurred  cumulative_paid
GRNAME              AccidentYear lag                                      
Allstate Ins Co Grp 1988         1                 367404            70571
                                 2                 362988           155905
                                 3                 347288           220744
                                 4                 330648           251595
                                 5                 354690           274156
...                                                   ...              ...
Tower Ins Co Of NY  1997         6                    334              287
                                 7                    318              293
                                 8                    323              300
                                 9                    310              297
                                 10                   312              297

[13200 rows x 2 columns]

Use `iloc()`

In [ ]:
print(df_triangle.iloc[:, 0])
print(df_triangle.iloc[:, :1])

## Row-wise subsets

In [ ]:
df_triangle[df_triangle['paid_to_incurred'] > 2]

In [ ]:
df_triangle[df_triangle['paid_to_incurred'] > 2]['cumulative_paid']

In [ ]:
df_triangle[df_triangle['paid_to_incurred'] > 2][['cumulative_paid']]

In [ ]:
df_triangle[df_triangle['paid_to_incurred'] > 2].cumulative_paid

In [ ]:
df_triangle[df_triangle['paid_to_incurred'] > 2]['cumulative_paid', 'cumulative_incurred']

In [ ]:
df_triangle[df_triangle['paid_to_incurred'] > 2][['cumulative_paid', 'cumulative_incurred']]

In [ ]:
df_triangle.loc[df_triangle['paid_to_incurred'] > 2, ['cumulative_paid', 'cumulative_incurred']]

In [ ]:
df_triangle.loc(df_triangle['AccidentYear'] <= 1989) #bc it's index

We could use `filter()` here, but I'm not wild about that.

In [ ]:
df_triangle.query('AccidentYear <= 1989 & lag == 1')['cumulative_paid']

In [ ]:
df_triangle.query('AccidentYear <= 1989 & lag == 1')[['cumulative_paid']] #Joseph added

In [112]:
df_upper = df_triangle.query('DevelopmentYear <= 1997') 
df_upper.shape

(7260, 13)

In [113]:
df_triangle.shape



(13200, 13)

In [114]:
df_upper.shape[0] / 55 #55 elements in upper triangle

132.0

You may be tempted by the `filter()` method. This will filter based on index values.

In [ ]:
# df_triangle[1996].cumulative_paid

In [ ]:
# df_triangle.loc(1996,'cumulative_paid') #Joseph added

# Reshaping

## Pivoting

In [101]:
make_float = lambda x: "${:,.2f}".format(x)

In [115]:
df_allstate = df_upper.query('GRNAME == "Allstate Ins Co Grp"')
df_allstate

DevelopmentYear  GRCODE  DevelopmentLag  \
GRNAME              AccidentYear lag                                            
Allstate Ins Co Grp 1988         1               1988      86               1   
                                 2               1989      86               2   
                                 3               1990      86               3   
                                 4               1991      86               4   
                                 5               1992      86               5   
                                 6               1993      86               6   
                                 7               1994      86               7   
                                 8               1995      86               8   
                                 9               1996      86               9   
                                 10              1997      86              10   
                    1989         1               1989      86               1   
                                 2               1990      86               2   
                                 3               1991      86               3   
                                 4               1992      86               4   
                                 5               1993      86               5   
                                 6               1994      86               6   
                                 7               1995      86               7   
                                 8               1996      86               8   
                                 9               1997      86               9   
                    1990         1               1990      86               1   
                                 2               1991      86               2   
                                 3               1992      86               3   
                                 4               1993      86               4   
                                 5               1994      86               5   
                                 6               1995      86               6   
                                 7               1996      86               7   
                                 8               1997      86               8   
                    1991         1               1991      86               1   
                                 2               1992      86               2   
                                 3               1993      86               3   
                                 4               1994      86               4   
                                 5               1995      86               5   
                                 6               1996      86               6   
                                 7               1997      86               7   
                    1992         1               1992      86               1   
                                 2               1993      86               2   
                                 3               1994      86               3   
                                 4               1995      86               4   
                                 5               1996      86               5   
                                 6               1997      86               6   
                    1993         1               1993      86               1   
                                 2               1994      86               2   
                                 3               1995      86               3   
                                 4               1996      86               4   
                                 5               1997      86               5   
                    1994         1               1994      86               1   
                                 2               1995      86               2   
                                 3               1996      86        

`pivot_table(values, index, columns)`

In [116]:
#df_allstate['cumulative_paid']
df_allstate.pivot_table(values='cumulative_paid',index = 'AccidentYear', columns = 'lag') #.apply(make_float)

lag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,70571.0,155905.0,220744.0,251595.0,274156.0,287676.0,298499.0,304873.0,321808.0,325322.0
1989,66547.0,136447.0,179142.0,211343.0,231430.0,244750.0,254557.0,270059.0,273873.0,NaN
1990,52233.0,133370.0,178444.0,204442.0,222193.0,232940.0,253337.0,256788.0,NaN,NaN
1991,59315.0,128051.0,169793.0,196685.0,213165.0,234676.0,239195.0,NaN,NaN,NaN
1992,39991.0,89873.0,114117.0,133003.0,154362.0,159496.0,NaN,NaN,NaN,NaN
1993,19744.0,47229.0,61909.0,85099.0,87215.0,NaN,NaN,NaN,NaN,NaN
1994,20379.0,46773.0,88636.0,91077.0,NaN,NaN,NaN,NaN,NaN,NaN
1995,18756.0,84712.0,87311.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,42609.0,44916.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
df_wide_paid = df_allstate.pivot_table('cumulative_paid', 'AccidentYear', 'lag')
df_wide_paid

lag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,70571.0,155905.0,220744.0,251595.0,274156.0,287676.0,298499.0,304873.0,321808.0,325322.0
1989,66547.0,136447.0,179142.0,211343.0,231430.0,244750.0,254557.0,270059.0,273873.0,NaN
1990,52233.0,133370.0,178444.0,204442.0,222193.0,232940.0,253337.0,256788.0,NaN,NaN
1991,59315.0,128051.0,169793.0,196685.0,213165.0,234676.0,239195.0,NaN,NaN,NaN
1992,39991.0,89873.0,114117.0,133003.0,154362.0,159496.0,NaN,NaN,NaN,NaN
1993,19744.0,47229.0,61909.0,85099.0,87215.0,NaN,NaN,NaN,NaN,NaN
1994,20379.0,46773.0,88636.0,91077.0,NaN,NaN,NaN,NaN,NaN,NaN
1995,18756.0,84712.0,87311.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,42609.0,44916.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
df_wide_paid.shape

(10, 10)

In [119]:
df_wide_paid.columns

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], dtype='int64', name='lag')

In [120]:
df_wide_paid.index

Int64Index([1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997], dtype='int64', name='AccidentYear')

Let's also construct an incurred triangle

In [121]:
df_wide_incurred = df_allstate.pivot_table('cumulative_incurred', 'AccidentYear', 'lag')
df_wide_incurred

lag,1,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,,
1988,367404.0,362988.0,347288.0,330648.0,354690.0,350092.0,346808.0,349124.0,348157.0,347762.0
1989,336928.0,316483.0,278496.0,303033.0,299496.0,295061.0,299251.0,297492.0,300620.0,NaN
1990,289198.0,311381.0,277980.0,277732.0,276563.0,278067.0,276704.0,281101.0,NaN,NaN
1991,297174.0,277209.0,269739.0,272666.0,271318.0,267578.0,269592.0,NaN,NaN,NaN
1992,181796.0,205079.0,199106.0,187833.0,185663.0,184940.0,NaN,NaN,NaN,NaN
1993,114807.0,114774.0,101460.0,98430.0,96930.0,NaN,NaN,NaN,NaN,NaN
1994,107934.0,107569.0,97730.0,96185.0,NaN,NaN,NaN,NaN,NaN,NaN
1995,100686.0,94456.0,92314.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,53381.0,51205.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Unstack

Unstack will behave similarly to `pivot_table()`, however it relies on values in the multiindex.

In [126]:
df_allstate.drop(columns='lagmo',inplace = True)
df_allstate.unstack() #use the right most index

C:\Users\e015614\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


DevelopmentYear                          \
lag                                           1       2       3       4    
GRNAME              AccidentYear                                           
Allstate Ins Co Grp 1988                  1988.0  1989.0  1990.0  1991.0   
                    1989                  1989.0  1990.0  1991.0  1992.0   
                    1990                  1990.0  1991.0  1992.0  1993.0   
                    1991                  1991.0  1992.0  1993.0  1994.0   
                    1992                  1992.0  1993.0  1994.0  1995.0   
                    1993                  1993.0  1994.0  1995.0  1996.0   
                    1994                  1994.0  1995.0  1996.0  1997.0   
                    1995                  1995.0  1996.0  1997.0     NaN   
                    1996                  1996.0  1997.0     NaN     NaN   
                    1997                  1997.0     NaN     NaN     NaN   

                                                                          \
lag                                   5       6       7       8       9    
GRNAME              AccidentYear                                           
Allstate Ins Co Grp 1988          1992.0  1993.0  1994.0  1995.0  1996.0   
                    1989          1993.0  1994.0  1995.0  1996.0  1997.0   
                    1990          1994.0  1995.0  1996.0  1997.0     NaN   
                    1991          1995.0  1996.0  1997.0     NaN     NaN   
                    1992          1996.0  1997.0     NaN     NaN     NaN   
                    1993          1997.0     NaN     NaN     NaN     NaN   
                    1994             NaN     NaN     NaN     NaN     NaN   
                    1995             NaN     NaN     NaN     NaN     NaN   
                    1996             NaN     NaN     NaN     NaN     NaN   
                    1997             NaN     NaN     NaN     NaN     NaN   

                                          ... paid_to_incurred            \
lag                                   10  ...               1         2    
GRNAME              AccidentYear          ...                              
Allstate Ins Co Grp 1988          1997.0  ...         0.192080  0.429505   
                    1989             NaN  ...         0.197511  0.431135   
                    1990             NaN  ...         0.180613  0.428318   
                    1991             NaN  ...         0.199597  0.461929   
                    1992             NaN  ...         0.219977  0.438236   
                    1993             NaN  ...         0.171976  0.411496   
                    1994             NaN  ...         0.188810  0.434819   
                    1995             NaN  ...         0.186282  0.896841   
                    1996             NaN  ...         0.798205  0.877180   
                    1997             NaN  ...         0.102751       NaN   

                                                                          \
lag                                     3         4         5         6    
GRNAME              AccidentYear                                           
Allstate Ins Co Grp 1988          0.635622  0.760915  0.772945  0.821715   
                    1989          0.643248  0.697426  0.772732  0.829489   
                    1990          0.641931  0.736113  0.803408  0.837712   
                    1991          0.629471  0.721340  0.785665  0.877038   
                    1992          0.573147  0.708092  0.831410  0.862420   
                    1993          0.610181  0.864564  0.899773       NaN   
                    1994          0.906948  0.946894       NaN       NaN   
                    1995          0.945805       NaN       NaN       NaN   
                    1996               NaN       NaN       NaN       NaN   
                    1997               NaN       NaN       NaN       NaN   

                                                                          
la

In [127]:
df_allstate[['cumulative_paid', 'cumulative_incurred']].unstack()

cumulative_paid                      \
lag                                           1         2         3    
GRNAME              AccidentYear                                       
Allstate Ins Co Grp 1988                 70571.0  155905.0  220744.0   
                    1989                 66547.0  136447.0  179142.0   
                    1990                 52233.0  133370.0  178444.0   
                    1991                 59315.0  128051.0  169793.0   
                    1992                 39991.0   89873.0  114117.0   
                    1993                 19744.0   47229.0   61909.0   
                    1994                 20379.0   46773.0   88636.0   
                    1995                 18756.0   84712.0   87311.0   
                    1996                 42609.0   44916.0       NaN   
                    1997                   691.0       NaN       NaN   

                                                                          \
lag                                     4         5         6         7    
GRNAME              AccidentYear                                           
Allstate Ins Co Grp 1988          251595.0  274156.0  287676.0  298499.0   
                    1989          211343.0  231430.0  244750.0  254557.0   
                    1990          204442.0  222193.0  232940.0  253337.0   
                    1991          196685.0  213165.0  234676.0  239195.0   
                    1992          133003.0  154362.0  159496.0       NaN   
                    1993           85099.0   87215.0       NaN       NaN   
                    1994           91077.0       NaN       NaN       NaN   
                    1995               NaN       NaN       NaN       NaN   
                    1996               NaN       NaN       NaN       NaN   
                    1997               NaN       NaN       NaN       NaN   

                                                                \
lag                                     8         9         10   
GRNAME              AccidentYear                                 
Allstate Ins Co Grp 1988          304873.0  321808.0  325322.0   
                    1989          270059.0  273873.0       NaN   
                    1990          256788.0       NaN       NaN   
                    1991               NaN       NaN       NaN   
                    1992               NaN       NaN       NaN   
                    1993               NaN       NaN       NaN   
                    1994               NaN       NaN       NaN   
                    1995               NaN       NaN       NaN   
                    1996               NaN       NaN       NaN   
                    1997               NaN       NaN       NaN   

                                 cumulative_incurred                      \
lag                                               1         2         3    
GRNAME              AccidentYear                                           
Allstate Ins Co Grp 1988                    367404.0  362988.0  347288.0   
                    1989                    336928.0  316483.0  278496.0   
                    1990                    289198.0  311381.0  277980.0   
                    1991                    297174.0  277209.0  269739.0   
                    1992                    181796.0  205079.0  199106.0   
                    1993                    114807.0  114774.0  101460.0   
                    1994                    107934.0  107569.0   97730.0   
                    1995                    100686.0   94456.0   92314.0   
                    1996                     53381.0   51205.0       NaN   
                    1997                      6725.0       NaN       NaN   

                                                                          \
lag                                     4         5         6         7    
GRNAME              AccidentYear                                           
Allstate Ins Co Grp 1988       

## Stack

In [128]:
df_wide_paid.stack()

AccidentYear  lag
1988          1       70571.0
              2      155905.0
              3      220744.0
              4      251595.0
              5      274156.0
              6      287676.0
              7      298499.0
              8      304873.0
              9      321808.0
              10     325322.0
1989          1       66547.0
              2      136447.0
              3      179142.0
              4      211343.0
              5      231430.0
              6      244750.0
              7      254557.0
              8      270059.0
              9      273873.0
1990          1       52233.0
              2      133370.0
              3      178444.0
              4      204442.0
              5      222193.0
              6      232940.0
              7      253337.0
              8      256788.0
1991          1       59315.0
              2      128051.0
              3      169793.0
              4      196685.0
              5      213165.0
              6      2

Notice that we dropped the NA values. We can keep them if we like.

In [129]:
df_wide_paid.stack(dropna = False)

AccidentYear  lag
1988          1       70571.0
              2      155905.0
              3      220744.0
              4      251595.0
              5      274156.0
                       ...   
1997          6           NaN
              7           NaN
              8           NaN
              9           NaN
              10          NaN
Length: 100, dtype: float64

In [130]:
df_long_paid = df_wide_paid.stack()
df_long_paid

AccidentYear  lag
1988          1       70571.0
              2      155905.0
              3      220744.0
              4      251595.0
              5      274156.0
              6      287676.0
              7      298499.0
              8      304873.0
              9      321808.0
              10     325322.0
1989          1       66547.0
              2      136447.0
              3      179142.0
              4      211343.0
              5      231430.0
              6      244750.0
              7      254557.0
              8      270059.0
              9      273873.0
1990          1       52233.0
              2      133370.0
              3      178444.0
              4      204442.0
              5      222193.0
              6      232940.0
              7      253337.0
              8      256788.0
1991          1       59315.0
              2      128051.0
              3      169793.0
              4      196685.0
              5      213165.0
              6      2

In [131]:
df_long_paid = df_wide_paid.stack().to_frame()
df_long_paid.columns = ['cumulative_paid']

In [132]:
df_long_incurred = df_wide_incurred.stack().to_frame()
df_long_incurred.columns = ['cumulative_incurred']

In [133]:
df_long_paid

cumulative_paid
AccidentYear lag                 
1988         1            70571.0
             2           155905.0
             3           220744.0
             4           251595.0
             5           274156.0
             6           287676.0
             7           298499.0
             8           304873.0
             9           321808.0
             10          325322.0
1989         1            66547.0
             2           136447.0
             3           179142.0
             4           211343.0
             5           231430.0
             6           244750.0
             7           254557.0
             8           270059.0
             9           273873.0
1990         1            52233.0
             2           133370.0
             3           178444.0
             4           204442.0
             5           222193.0
             6           232940.0
             7           253337.0
             8           256788.0
1991         1            59315.0
             2           128051.0
             3           169793.0
             4           196685.0
             5           213165.0
             6           234676.0
             7           239195.0
1992         1            39991.0
             2            89873.0
             3           114117.0
             4           133003.0
             5           154362.0
             6           159496.0
1993         1            19744.0
             2            47229.0
             3            61909.0
             4            85099.0
             5            87215.0
1994         1            20379.0
             2            46773.0
             3            88636.0
             4            91077.0
1995         1            18756.0
             2            84712.0
             3            87311.0
1996         1            42609.0
             2            44916.0
1997         1              691.0

In [134]:
df_long_incurred

cumulative_incurred
AccidentYear lag                     
1988         1               367404.0
             2               362988.0
             3               347288.0
             4               330648.0
             5               354690.0
             6               350092.0
             7               346808.0
             8               349124.0
             9               348157.0
             10              347762.0
1989         1               336928.0
             2               316483.0
             3               278496.0
             4               303033.0
             5               299496.0
             6               295061.0
             7               299251.0
             8               297492.0
             9               300620.0
1990         1               289198.0
             2               311381.0
             3               277980.0
             4               277732.0
             5               276563.0
             6               278067.0
             7               276704.0
             8               281101.0
1991         1               297174.0
             2               277209.0
             3               269739.0
             4               272666.0
             5               271318.0
             6               267578.0
             7               269592.0
1992         1               181796.0
             2               205079.0
             3               199106.0
             4               187833.0
             5               185663.0
             6               184940.0
1993         1               114807.0
             2               114774.0
             3               101460.0
             4                98430.0
             5                96930.0
1994         1               107934.0
             2               107569.0
             3                97730.0
             4                96185.0
1995         1               100686.0
             2                94456.0
             3                92314.0
1996         1                53381.0
             2                51205.0
1997         1                 6725.0

## Merge two data frames

In [ ]:
df_new = pd.merge(df_long_paid, df_long_incurred) #Need to specify if merge by index

In [135]:
df_new = pd.merge(df_long_paid, df_long_incurred, left_index = True, right_index = True)
df_new

cumulative_paid  cumulative_incurred
AccidentYear lag                                      
1988         1            70571.0             367404.0
             2           155905.0             362988.0
             3           220744.0             347288.0
             4           251595.0             330648.0
             5           274156.0             354690.0
             6           287676.0             350092.0
             7           298499.0             346808.0
             8           304873.0             349124.0
             9           321808.0             348157.0
             10          325322.0             347762.0
1989         1            66547.0             336928.0
             2           136447.0             316483.0
             3           179142.0             278496.0
             4           211343.0             303033.0
             5           231430.0             299496.0
             6           244750.0             295061.0
             7           254557.0             299251.0
             8           270059.0             297492.0
             9           273873.0             300620.0
1990         1            52233.0             289198.0
             2           133370.0             311381.0
             3           178444.0             277980.0
             4           204442.0             277732.0
             5           222193.0             276563.0
             6           232940.0             278067.0
             7           253337.0             276704.0
             8           256788.0             281101.0
1991         1            59315.0             297174.0
             2           128051.0             277209.0
             3           169793.0             269739.0
             4           196685.0             272666.0
             5           213165.0             271318.0
             6           234676.0             267578.0
             7           239195.0             269592.0
1992         1            39991.0             181796.0
             2            89873.0             205079.0
             3           114117.0             199106.0
             4           133003.0             187833.0
             5           154362.0             185663.0
             6           159496.0             184940.0
1993         1            19744.0             114807.0
             2            47229.0             114774.0
             3            61909.0             101460.0
             4            85099.0              98430.0
             5            87215.0              96930.0
1994         1            20379.0             107934.0
             2            46773.0             107569.0
             3            88636.0              97730.0
             4            91077.0              96185.0
1995         1            18756.0             100686.0
             2            84712.0              94456.0
             3            87311.0              92314.0
1996         1            42609.0              53381.0
             2            44916.0              51205.0
1997         1              691.0               6725.0

# Group-wise operations

In [136]:
df_allstate['cumulative_paid'].shift()

GRNAME               AccidentYear  lag
Allstate Ins Co Grp  1988          1           NaN
                                   2       70571.0
                                   3      155905.0
                                   4      220744.0
                                   5      251595.0
                                   6      274156.0
                                   7      287676.0
                                   8      298499.0
                                   9      304873.0
                                   10     321808.0
                     1989          1      325322.0
                                   2       66547.0
                                   3      136447.0
                                   4      179142.0
                                   5      211343.0
                                   6      231430.0
                                   7      244750.0
                                   8      254557.0
                                   9      2

In [137]:
df_allstate['prior_cumulative_paid'] = df_allstate['cumulative_paid'].shift()

C:\Users\e015614\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [138]:
df_allstate[['cumulative_paid', 'prior_cumulative_paid']].head(15)

cumulative_paid  prior_cumulative_paid
GRNAME              AccidentYear lag                                        
Allstate Ins Co Grp 1988         1              70571                    NaN
                                 2             155905                70571.0
                                 3             220744               155905.0
                                 4             251595               220744.0
                                 5             274156               251595.0
                                 6             287676               274156.0
                                 7             298499               287676.0
                                 8             304873               298499.0
                                 9             321808               304873.0
                                 10            325322               321808.0
                    1989         1              66547               325322.0
                                 2             136447                66547.0
                                 3             179142               136447.0
                                 4             211343               179142.0
                                 5             231430               211343.0

We have a problem. The entry for 1989, lag 1 is not correct. We need to group by accident year

In [139]:
df_allstate['prior_cumulative_paid'] = df_allstate['cumulative_paid'].groupby(
    level='AccidentYear'
  ).apply(lambda  x : x.shift(1))

C:\Users\e015614\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


The warning is something we should pay attention to, but it's not anything to worry about in this case.

In [140]:
df_allstate[['cumulative_paid', 'prior_cumulative_paid']].head(15)

cumulative_paid  prior_cumulative_paid
GRNAME              AccidentYear lag                                        
Allstate Ins Co Grp 1988         1              70571                    NaN
                                 2             155905                70571.0
                                 3             220744               155905.0
                                 4             251595               220744.0
                                 5             274156               251595.0
                                 6             287676               274156.0
                                 7             298499               287676.0
                                 8             304873               298499.0
                                 9             321808               304873.0
                                 10            325322               321808.0
                    1989         1              66547                    NaN
                                 2             136447                66547.0
                                 3             179142               136447.0
                                 4             211343               179142.0
                                 5             231430               211343.0

Do that again for the incurred

In [141]:
df_allstate['prior_cumulative_incurred'] = df_allstate['cumulative_incurred'].groupby(
    level='AccidentYear'
  ).apply(lambda  x : x.shift(1))

C:\Users\e015614\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [143]:
cols =[col for col in df_allstate.colums]
cols
#df_allstate[cols]

AttributeError: 'DataFrame' object has no attribute 'colums'

## Make some link ratios

In [144]:
df_allstate['paid_ldf'] = df_allstate['cumulative_paid'] / df_allstate['prior_cumulative_paid']
df_allstate['incurred_ldf'] = df_allstate['cumulative_incurred'] / df_allstate['prior_cumulative_incurred']

C:\Users\e015614\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\e015614\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [146]:
df_allstate[['cumulative_paid','paid_ldf', 'prior_cumulative_paid', 'incurred_ldf']]

cumulative_paid  paid_ldf  \
GRNAME              AccidentYear lag                              
Allstate Ins Co Grp 1988         1              70571       NaN   
                                 2             155905  2.209194   
                                 3             220744  1.415888   
                                 4             251595  1.139759   
                                 5             274156  1.089672   
                                 6             287676  1.049315   
                                 7             298499  1.037622   
                                 8             304873  1.021354   
                                 9             321808  1.055548   
                                 10            325322  1.010920   
                    1989         1              66547       NaN   
                                 2             136447  2.050385   
                                 3             179142  1.312905   
                                 4             211343  1.179751   
                                 5             231430  1.095045   
                                 6             244750  1.057555   
                                 7             254557  1.040069   
                                 8             270059  1.060898   
                                 9             273873  1.014123   
                    1990         1              52233       NaN   
                                 2             133370  2.553367   
                                 3             178444  1.337962   
                                 4             204442  1.145693   
                                 5             222193  1.086827   
                                 6             232940  1.048368   
                                 7             253337  1.087563   
                                 8             256788  1.013622   
                    1991         1              59315       NaN   
                                 2             128051  2.158830   
                                 3             169793  1.325979   
                                 4             196685  1.158381   
                                 5             213165  1.083789   
                                 6             234676  1.100912   
                                 7             239195  1.019256   
                    1992         1              39991       NaN   
                                 2              89873  2.247331   
                                 3             114117  1.269758   
                                 4             133003  1.165497   
                                 5             154362  1.160590   
                                 6             159496  1.033259   
                    1993         1              19744       NaN   
                                 2              47229  2.392068   
                                 3              61909  1.310826   
                                 4              85099  1.374582   
                                 5              87215  1.024865   
                    1994         1              20379       NaN   
                                 2              46773  2.295157   
                                 3              88636  1.895025   
                                 4              91077  1.027540   
                    1995         1              18756       NaN   
                                 2              84712  4.516528   
                                 3              87311  1.030680   
                    1996         1              42609       NaN   
                                 2              44916  1.054143   
                    1997         1                691       NaN   

                                      prior_cumulative_paid  incurred_ldf  
GRNAME              AccidentYear lag                                       
Allstate Ins Co Grp 1988         1                      NaN       

In [147]:
df_allstate.pivot_table(index = 'AccidentYear', columns = 'lag', values = 'paid_ldf').round(3)

lag,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,
1988,2.209,1.416,1.140,1.090,1.049,1.038,1.021,1.056,1.011
1989,2.050,1.313,1.180,1.095,1.058,1.040,1.061,1.014,NaN
1990,2.553,1.338,1.146,1.087,1.048,1.088,1.014,NaN,NaN
1991,2.159,1.326,1.158,1.084,1.101,1.019,NaN,NaN,NaN
1992,2.247,1.270,1.165,1.161,1.033,NaN,NaN,NaN,NaN
1993,2.392,1.311,1.375,1.025,NaN,NaN,NaN,NaN,NaN
1994,2.295,1.895,1.028,NaN,NaN,NaN,NaN,NaN,NaN
1995,4.517,1.031,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,1.054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [148]:
df_allstate.pivot_table(index = 'AccidentYear', columns = 'lag', values = 'paid_ldf').round(3).fillna("")

lag,2,3,4,5,6,7,8,9,10
AccidentYear,,,,,,,,,
1988,2.209,1.416,1.14,1.09,1.049,1.038,1.021,1.056,1.011
1989,2.050,1.313,1.18,1.095,1.058,1.04,1.061,1.014,
1990,2.553,1.338,1.146,1.087,1.048,1.088,1.014,,
1991,2.159,1.326,1.158,1.084,1.101,1.019,,,
1992,2.247,1.27,1.165,1.161,1.033,,,,
1993,2.392,1.311,1.375,1.025,,,,,
1994,2.295,1.895,1.028,,,,,,
1995,4.517,1.031,,,,,,,
1996,1.054,,,,,,,,


## Weighted average link ratios

In [149]:
cumul_cols = [col_name for col_name in df_allstate.columns if 'cumulative' in col_name] #Grouping by lags
df_links = df_allstate.groupby(level='lag')[cumul_cols].sum()
df_links

,cumulative_incurred,cumulative_paid,prior_cumulative_paid,prior_cumulative_incurred
lag,,,,
1,1856033,390836,0.0,0.0
2,1841144,867276,390145.0,1849308.0
3,1664113,1100096,822360.0,1789939.0
4,1566527,1173244,1012785.0,1571799.0
5,1484660,1182521,1082167.0,1470342.0
6,1375738,1159538,1095306.0,1387730.0
7,1192355,1045588,1000042.0,1190798.0
8,927717,831720,806393.0,922763.0
9,648777,595681,574932.0,646616.0


In [150]:
df_links['paid_ata'] = df_links.cumulative_paid / df_links.prior_cumulative_paid
df_links['incurred_ata'] = df_links.cumulative_incurred / df_links.prior_cumulative_incurred
df_links

,cumulative_incurred,cumulative_paid,prior_cumulative_paid,prior_cumulative_incurred,paid_ata,incurred_ata
lag,,,,,,
1,1856033,390836,0.0,0.0,inf,inf
2,1841144,867276,390145.0,1849308.0,2.222958,0.995585
3,1664113,1100096,822360.0,1789939.0,1.337730,0.929704
4,1566527,1173244,1012785.0,1571799.0,1.158433,0.996646
5,1484660,1182521,1082167.0,1470342.0,1.092734,1.009738
6,1375738,1159538,1095306.0,1387730.0,1.058643,0.991359
7,1192355,1045588,1000042.0,1190798.0,1.045544,1.001308
8,927717,831720,806393.0,922763.0,1.031408,1.005369
9,648777,595681,574932.0,646616.0,1.036089,1.003342


In [151]:
df_links = df_links.query('lag > 1') #Not needed in table
df_links

,cumulative_incurred,cumulative_paid,prior_cumulative_paid,prior_cumulative_incurred,paid_ata,incurred_ata
lag,,,,,,
2,1841144,867276,390145.0,1849308.0,2.222958,0.995585
3,1664113,1100096,822360.0,1789939.0,1.337730,0.929704
4,1566527,1173244,1012785.0,1571799.0,1.158433,0.996646
5,1484660,1182521,1082167.0,1470342.0,1.092734,1.009738
6,1375738,1159538,1095306.0,1387730.0,1.058643,0.991359
7,1192355,1045588,1000042.0,1190798.0,1.045544,1.001308
8,927717,831720,806393.0,922763.0,1.031408,1.005369
9,648777,595681,574932.0,646616.0,1.036089,1.003342
10,347762,325322,321808.0,348157.0,1.010920,0.998865


In [ ]:
df_links.paid_ata.cumprod() #there is a problem

In [152]:
df_links.paid_ata[::-1].cumprod() #Reverse before cumulative product (reverse order but index is in correct order)

lag
10    1.010920
9     1.047403
8     1.080300
7     1.129501
6     1.195738
5     1.306624
4     1.513637
3     2.024839
2     4.501131
Name: paid_ata, dtype: float64

In [153]:
df_links['paid_atu'] = df_links.paid_ata[::-1].cumprod()
df_links['incurred_atu'] = df_links.incurred_ata[::-1].cumprod()

C:\Users\e015614\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\e015614\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_links

In [ ]:
df_links = df_links[['paid_atu', 'incurred_atu']]
df_links

In [ ]:
df_links = df_links.reset_index() #Readjust the index
df_links.lag = df_links.lag - 1
df_links = df_links.set_index('lag')
df_links
#Or df_links.index - 1

In [ ]:
df_ultimate = df_allstate.query('DevelopmentYear == 1997')
df_ultimate

In [ ]:
df_ultimate = pd.merge(df_ultimate, df_links, left_index = True, right_index = True)
df_ultimate['ult_paid'] = df_ultimate.cumulative_paid * df_ultimate.paid_atu
df_ultimate['ult_incurred'] = df_ultimate.cumulative_incurred * df_ultimate.incurred_atu
df_ultimate[['ult_paid', 'ult_incurred']]

In [ ]:
df_ultimate[['ult_paid', 'paid_atu']]

In [ ]:
df_ultimate[[ 'ult_incurred', 'incurred_atu']]

# Saving our work

In [ ]:
import os
data= os.getcwd()
data

In [ ]:
!dir

In [ ]:
df_triangle.to_csv(data + '/df_ultimate.csv')

# If there's time

## Recap on generator objects and iteration

# Homework

1. Repeat the construction of LDFs for every company. 
2. Which company had the most significant difference between paid and incurred ultimate estimates?
1. Which company had the largest case reserves? In which cell can you find this?